In [ ]:
import os
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

**Until now** In our previous folder **(Data Cleaning and visualization)**<br>
1:-We found that **(150,150) is the best image size for our image**.<br>
2:-We remove corrupted images.<br>
3:-Split our data into train and validation<br>
4:-Balanced our Imbalanced training dataset with the help of **Image Data Augmentation.**

In this folder **(Base model)** work we doing.<br>

1:- Generate our data by using **ImageDataGenerator**.<br>
2:- Resacle and resize our data in **ImagedataGenerator**.<br>
3:- Built our **Model Base**.<br>

And other process.

In [ ]:
print(len(os.listdir('cats-v-dogs/training/cats/')))
print(len(os.listdir('cats-v-dogs/training/dogs/')))

In [ ]:
train = "cats-v-dogs/training/"
validation = "cats-v-dogs/testing/"

# Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.) #rescale
train_data = train_datagen.flow_from_directory(train,
                                              batch_size = 20,
                                              class_mode = "binary",
                                              target_size = (150,150)) # resize

validation_datagen = ImageDataGenerator(rescale = 1.0/255.) # rescale
validaton_data = validation_datagen.flow_from_directory(validation,
                                                       batch_size = 20,
                                                       class_mode = "binary",
                                                       target_size = (150,150)) # resize

# Base Model

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),padding="same",input_shape = (150,150,3)))
model.add(layers.ReLU()) 
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(32,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(64,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(layers.Dense(64))
model.add(layers.ReLU())

model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))


In [ ]:
model.summary()

In [ ]:
from keras.optimizers import RMSprop
model.compile(loss= "binary_crossentropy",
             optimizer =RMSprop(lr=0.001),
             metrics=["accuracy"])

In [ ]:
history = model.fit_generator(train_data,
                              epochs = 30, 
                              steps_per_epoch = 21147//20, # images = batch_size * steps
                              validation_data = validaton_data,
                              validation_steps=2342//20) # images = batch_size * steps

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

As we see our base model is overfitting.<br>
There are three cause of overfitting:- <br>
1. Lack of data.
2. Unbalanced data (but we already solve this cause).
3. Our model is not much complex for this model.

Solution:-<br> To overcome overfitting first we use data augmentation and then train our model.


# new model

# Data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train_data = train_datagen.flow_from_directory(train,
                                              batch_size = 150,
                                              class_mode = "binary",
                                              target_size = (150,150)) 

validation_datagen = ImageDataGenerator(rescale = 1.0/255.) # rescale
validaton_data = validation_datagen.flow_from_directory(validation,
                                                       batch_size = 150,
                                                       class_mode = "binary",
                                                       target_size = (150,150)) # resize

# Model

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),padding="same",input_shape = (150,150,3)))
model.add(layers.ReLU()) 
model.add(layers.MaxPooling2D(pool_size=(2,2)))
  
model.add(layers.Conv2D(64,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(128,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(128,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(layers.Dense(512))
model.add(layers.ReLU())

model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import RMSprop
model.compile(loss= "binary_crossentropy",
             optimizer =RMSprop(lr=0.0001),
             metrics=["accuracy"])

In [ ]:
history = model.fit_generator(train_data,
                  epochs = 30, 
                 steps_per_epoch = 21147//150, # images = batch_size * steps
                validation_data = validaton_data,
                validation_steps=2342//150) # images = batch_size * steps )

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

After increasing image data by using Data Augmentation, our model predict well good but little underfitting.<br>
Now we use regularization technique (Dropout) with 50% 0f neuron.

# New Model

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),padding="same",input_shape = (150,150,3)))
model.add(layers.ReLU()) 
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(64,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(128,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(128,(3,3),padding="same"))
model.add(layers.ReLU())
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Dropout(0.5))

model.add(layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(layers.Dense(512))
model.add(layers.ReLU())

model.add(layers.Dropout(0.2))

model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train_data = train_datagen.flow_from_directory(train,
                                              batch_size = 150,
                                              class_mode = "binary",
                                              target_size = (150,150)) 

validation_datagen = ImageDataGenerator(rescale = 1.0/255.) # rescale
validaton_data = validation_datagen.flow_from_directory(validation,
                                                       batch_size = 150,
                                                       class_mode = "binary",
                                                       target_size = (150,150)) # resize

In [ ]:
from keras.optimizers import RMSprop
model.compile(loss= "binary_crossentropy",
             optimizer =RMSprop(lr=0.0001),
             metrics=["accuracy"])

In [ ]:
history = model.fit_generator(train_data,
                  epochs = 30, 
                 steps_per_epoch = 21147//150, # (images = batch_size * steps)
                validation_data = validaton_data,  
                validation_steps=2342//150) # (images = batch_size * steps)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show() 

In [ ]:
model.save("cat_dog.h5") 

Little bit good but i think it need more Dropout.

# Prediction

In [ ]:
import cv2
imgplot = cv2.imread("test1.jpg")

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

model = load_model('cat_dog.h5')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('test1.jpg') #dog
img = cv2.resize(img,(150,150))
img = img.astype('float32')
img = np.reshape(img,[1,150,150,3])

classes = model.predict_classes(img)

print (classes)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

image_path="test/cat.jpg"
img = image.load_img(image_path, target_size=(150, 150))
plt.imshow(img)
img = np.expand_dims(img, axis=0)
result=model.predict_classes(img)
plt.title((result[0][0]))
plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

image_path="dog.jpg"
img = image.load_img(image_path, target_size=(150, 150))
plt.imshow(img)
img = np.expand_dims(img, axis=0)
result=model.predict_classes(img)
plt.title((result[0][0]))
plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

image_path="test/dog.jpg"
img = image.load_img(image_path, target_size=(150, 150))
plt.imshow(img)
img = np.expand_dims(img, axis=0)
result=model.predict_classes(img)
plt.title((result[0][0]))
plt.show()